In [1]:
# Eric Escalante's code:
import pandas as pd
from datetime import datetime

def get_data():
    df = pd.read_csv('http://python.zach.lol/access.log',          
                  engine='python',
                  header=None,
                  index_col=False,
                  names=['ip', 'timestamp', 'request_method', 'status', 'size', 'request_agent'],
                  sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])',
                  na_values='-',
                  usecols=[0, 3, 4, 5, 6, 8],
                  skip_blank_lines=True)

    df.timestamp = df.timestamp.str.replace('[', '')
    df.timestamp = df.timestamp.str.replace(']', '')
    df.timestamp= pd.to_datetime(df.timestamp.str.replace(':', ' ', 1)) 

    df.request_method = df.request_method.str.replace('"', '')

    df = df.set_index('timestamp')
    df = df.tz_localize('utc').tz_convert('America/Chicago')
    
    return(df)

if __name__ == '__main__':
    new_df = get_data()
    print(new_df.head(3).append(new_df.tail(3)))

                                     ip                      request_method  \
timestamp                                                                     
2019-04-16 14:34:42-05:00  97.105.19.58  GET /api/v1/sales?page=81 HTTP/1.1   
2019-04-16 14:34:42-05:00  97.105.19.58          GET /api/v1/items HTTP/1.1   
2019-04-16 14:34:44-05:00  97.105.19.58  GET /api/v1/sales?page=82 HTTP/1.1   
2019-04-17 07:55:13-05:00  97.105.19.58  GET /api/v1/sales?page=73 HTTP/1.1   
2019-04-17 07:55:13-05:00  97.105.19.58  GET /api/v1/sales?page=69 HTTP/1.1   
2019-04-17 07:55:14-05:00  97.105.19.58  GET /api/v1/sales?page=71 HTTP/1.1   

                           status    size             request_agent  
timestamp                                                            
2019-04-16 14:34:42-05:00     200  512495  "python-requests/2.21.0"  
2019-04-16 14:34:42-05:00     200    3561  "python-requests/2.21.0"  
2019-04-16 14:34:44-05:00     200  510103  "python-requests/2.21.0"  
2019-04-17 07:55:

In [ ]:
#!/usr/bin/env python
import gzip
import os
import sys
import re

# INPUT_DIR = "nginx-logs"
INPUT_DIR = "./"

lineformat = re.compile(r"""(?P<ipaddress>\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}) - - \[(?P<dateandtime>\d{2}\/[a-z]{3}\/\d{4}:\d{2}:\d{2}:\d{2} (\+|\-)\d{4})\] ((\"(GET|POST) )(?P<url>.+)(http\/1\.1")) (?P<statuscode>\d{3}) (?P<bytessent>\d+) (["](?P<refferer>(\-)|(.+))["]) (["](?P<useragent>.+)["])""", re.IGNORECASE)

for f in os.listdir(INPUT_DIR):
    if f.endswith(".gz"):
        logfile = gzip.open(os.path.join(INPUT_DIR, f))
    else:
        logfile = open(os.path.join(INPUT_DIR, f))

    for l in logfile.readlines():
        data = re.search(lineformat, l)
        if data:
            datadict = data.groupdict()
            ip = datadict["ipaddress"]
            datetimestring = datadict["dateandtime"]
            url = datadict["url"]
            bytessent = datadict["bytessent"]
            referrer = datadict["refferer"]
            useragent = datadict["useragent"]
            status = datadict["statuscode"]
            method = data.group(6)

            print (ip, \
              datetimestring, \
              url, \
              bytessent, \
              referrer, \
              useragent, \
              status, \
              method)

    logfile.close()

In [ ]:
import os
import sys
import re

# INPUT_DIR = "nginx-logs"
INPUT_DIR = "./"

lineformat = re.compile(r"""(?P<ipaddress>\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}) - - \[(?P<dateandtime>\d{2}\/[a-z]{3}\/\d{4}:\d{2}:\d{2}:\d{2} (\+|\-)\d{4})\] ((\"(GET|POST) )(?P<url>.+)(http\/1\.1")) (?P<statuscode>\d{3}) (?P<bytessent>\d+) (["](?P<refferer>(\-)|(.+))["]) (["](?P<useragent>.+)["])""", re.IGNORECASE)

os.listdir(INPUT_DIR)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
def process_line(line):
    # figure out how to deal with a single line
    ip, _, _, timestamp1, timestamp2, method, path, protocol, status, size, previous_page, *user_agent=line.split(' ')
    
    timestamp.str.replace('[', '').replace(']', '')
    timestamp1 = timestamp1.replace('[', '')
    timestamp2 = timestamp2.replace(']', '')
    timestamp = timestamp1 + timestamp2
#     timestamp.str.replace(':', ' ')
        
    method = method.replace('"', '')
    protocol = protocol.replace('"', '')
    previous_page = previous_page.replace('"', '')

    return dict(ip=ip, 
                timestamp=timestamp,
                method=method,
                path=path,
                protocol=protocol,
                code=status,
                size=size,
                previous_page=previous_page,
                user_agent=user_agent)


# read the file in
with open('./access.log') as f:
    contents = f.read()
    
# turn it into lines
lines = contents.split('\n')
lines = lines[:-1]
print(len(lines))

# apply that to all the lines
processed_lines = []

for line in lines:
    processed_lines.append(process_line(line))
    
df = pd.DataFrame(processed_lines)
df.head()

1. make sure timestamp is a datetime type (Hint:pd.to_datetime)
2. What does the unknown column mean?
3. What do you make of the user_agent data?
4. How many unique ip addresses are there? What does this mean? What is the most frequent ip? What does this mean?